**insert the testing and training images in seperate folders and pass the images to below given variables**

In [84]:
train_dir='C:/Users/rupa/anaconda3/envs/FinalProject_AutoML/sampleImages/covid/train_filtered/'
test_dir='C:/Users/rupa/anaconda3/envs/FinalProject_AutoML/sampleImages/covid/test_filtered/'
covidTrainingAuc=0
covidTestingAuc=0
pneumoniaTrainingAuc=0
pneumoniaTetingAuc=0

**Deducting Covid-19 **

In [85]:
import numpy as np
import pandas as pd 
from keras.layers import Input, Lambda, Dense, Flatten , Dropout , MaxPool2D
from keras.models import Model , Sequential
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from glob import glob
import matplotlib.pyplot as plt
import random
import os
from distutils.file_util import copy_file
import warnings
warnings.filterwarnings('ignore')
from keras.callbacks import EarlyStopping ,ReduceLROnPlateau
from IPython.display import Javascript

In [86]:
img_height = 64
img_width = 64
batch_size = 8
train_datagen = ImageDataGenerator(validation_split=0.2) # set validation split

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    classes = ['normal','covid'],
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    classes = ['normal','covid'],
    subset='validation')


test_generator = test_datagen.flow_from_directory(
    test_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    classes = ['normal','covid'],
    shuffle=False,
    class_mode='binary')

Found 28 images belonging to 2 classes.
Found 7 images belonging to 2 classes.
Found 35 images belonging to 2 classes.


In [87]:
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras import layers, metrics
from tensorflow.keras import callbacks
from tensorflow.keras.optimizers import Adam
from collections import Counter
from sklearn.metrics import roc_auc_score

def get_model(res, verbose=0):
    base_model = DenseNet121(input_shape=(res, res, 3),include_top=False,weights='imagenet',pooling='avg')
    x = base_model.output
    x = layers.Dense(128, activation="relu")(x)
    output = layers.Dense(1, activation="sigmoid")(x)
    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=1e-5),loss='binary_crossentropy',metrics=[metrics.AUC(name='auc')])
    if verbose: print(model.summary())
    return model

def get_class_weights(train_gen):
    counter = Counter(train_gen.classes)                          
    max_val = float(max(counter.values()))       
    class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}           
    print(f'class_weights {class_weights}')
    return class_weights

def train_model(train_gen, val_gen, model, class_weights, epochs):
    es_cb = callbacks.EarlyStopping(monitor='val_loss',patience=2,min_delta=0.001)
    lr_cb = callbacks.ReduceLROnPlateau(patience=1,min_delta=.05)

    history = model.fit(
        train_gen
        ,epochs=epochs
        ,validation_data=val_gen
        ,callbacks=[es_cb, lr_cb]
        ,class_weight=class_weights)
    
res = 64
model_densenet121 = get_model(res, verbose=0)
class_weights = get_class_weights(train_generator)
    

class_weights {0: 2.5, 1: 1.0}


In [88]:
def test_model(model, test_gen):
    y_pred_prob = model.predict(test_gen)
    y_true = test_gen.classes
    auc_score = roc_auc_score(y_true, y_pred_prob)
    return auc_score

In [89]:
#train_model(train_generator, validation_generator, model_densenet121, class_weights, epochs=5)
ex_model_densenet121 = model_densenet121.fit(train_generator,epochs=10)

Epoch 1/10
4/4 [==============================] - 43s 948ms/step - loss: 0.9041 - auc: 0.4914
Epoch 2/10
4/4 [==============================] - 3s 810ms/step - loss: 0.8031 - auc: 0.6195
Epoch 3/10
4/4 [==============================] - 3s 904ms/step - loss: 0.5898 - auc: 0.7333
Epoch 4/10
4/4 [==============================] - 3s 782ms/step - loss: 0.6645 - auc: 0.6987
Epoch 5/10
4/4 [==============================] - 3s 783ms/step - loss: 0.5282 - auc: 0.9272
Epoch 6/10
4/4 [==============================] - 3s 804ms/step - loss: 0.5677 - auc: 0.8514
Epoch 7/10
4/4 [==============================] - 3s 786ms/step - loss: 0.5558 - auc: 0.7234
Epoch 8/10
4/4 [==============================] - 3s 774ms/step - loss: 0.4203 - auc: 0.9388
Epoch 9/10
4/4 [==============================] - 3s 821ms/step - loss: 0.4314 - auc: 0.9470
Epoch 10/10
4/4 [==============================] - 3s 764ms/step - loss: 0.2648 - auc: 0.9975


In [90]:
covidTrainingAuc=ex_model_densenet121.history['auc'][9]*100
if covidTrainingAuc!=0:
    covidTestingAuc=test_model(model_densenet121, test_generator)*100


**Deducting Pneumonia **

In [91]:
BATCH_SIZE = 8
IMG_SIZE = 224
train_image_generator = ImageDataGenerator(rescale=1/255.)
test_image_generator = ImageDataGenerator(rescale=1/255.)
train_gen = train_image_generator.flow_from_directory(batch_size = BATCH_SIZE,
                                                directory= train_dir,
                                                shuffle = True,
                                                target_size = (IMG_SIZE, IMG_SIZE))
test_gen = test_image_generator.flow_from_directory(batch_size = BATCH_SIZE,
                                                directory= test_dir,
                                                target_size = (IMG_SIZE, IMG_SIZE))

Found 35 images belonging to 2 classes.
Found 35 images belonging to 2 classes.


In [92]:
for item in train_gen:
    images,labels = item
    print(images.shape)
    print(labels.shape)
    break
resnet50v2 = tf.keras.applications.ResNet50V2(include_top = False, 
                                  weights='imagenet', 
                                  input_shape = (IMG_SIZE, IMG_SIZE, 3))

(8, 224, 224, 3)
(8, 2)


In [93]:
model_resNet50  = tf.keras.Sequential([
        resnet50v2,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(2, activation='softmax')
])

model_resNet50.compile(loss='categorical_crossentropy', 
              metrics=[
                  tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                  tf.keras.metrics.Precision(name='precision'),
                  tf.keras.metrics.Recall(name='recall'),
              ], 
              optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-5))
model_resNet50.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 7, 7, 2048)        23564800  
_________________________________________________________________
flatten_2 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               12845184  
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 2)                

In [94]:
history = model_resNet50.fit(train_gen, epochs=5)

Epoch 1/5
5/5 [==============================] - 76s 5s/step - loss: 0.7720 - accuracy: 0.6705 - precision: 0.6770 - recall: 0.6514
Epoch 2/5
5/5 [==============================] - 28s 6s/step - loss: 0.4753 - accuracy: 0.6898 - precision: 0.6781 - recall: 0.6869
Epoch 3/5
5/5 [==============================] - 28s 5s/step - loss: 0.2461 - accuracy: 0.8173 - precision: 0.8236 - recall: 0.8101
Epoch 4/5
5/5 [==============================] - 27s 5s/step - loss: 0.2660 - accuracy: 0.8165 - precision: 0.7650 - recall: 0.9228
Epoch 5/5
5/5 [==============================] - 27s 5s/step - loss: 0.1017 - accuracy: 0.9699 - precision: 0.9753 - recall: 0.9651


In [95]:
scores = model_resNet50.evaluate(test_gen)
pneumoniaTrainingAuc=history.history['accuracy'][4]*100
pneumoniaTetingAuc=scores[1]*100
#print("\n%s: %.2f%%" % ("Testing Accuracy:", scores[1]*100))

5/5 [==============================] - 6s 820ms/step - loss: 0.5599 - accuracy: 0.7143 - precision: 0.6923 - recall: 0.7714


**deducting severity of the image**

In [96]:
if(covidTrainingAuc>pneumoniaTrainingAuc):
    print("Combined Accuracy of all images")
    print("Covid training Accuracy:",covidTrainingAuc)
    print("Covid testing Accuracy:",covidTestingAuc)
    path_new = (os.listdir(train_dir+"covid/"))
    #print(path_new)
    for filename in path_new:
        x = os.path.join(train_dir+"covid/", filename)
        y = str(x.replace("/","\\"))
        %run "predict_severity" $y
else:
    print("Combined Accuracy of all images")
    print("Pneumonia training Accuracy:",pneumoniaTrainingAuc)
    print("Pneumonia testing Accuracy:",pneumoniaTetingAuc)
    path_new = (os.listdir(train_dir+"pneumonia/"))
    #print(path_new)
    for filename in path_new:
        x = os.path.join(train_dir+"pneumonia/", filename)
        y = str(x.replace("/","\\"))
        %run "predict_severity" $y 

Combined Accuracy of all images
Covid training Accuracy: 99.37499761581421
Covid testing Accuracy: 82.80000000000001
Severity of the virus in  COVID-1638.jpg  X-Ray
Geographic Extent (0-8): 0.0
Opacity (0-8): 0.0
Severity of the virus in  COVID-1650.jpg  X-Ray
Geographic Extent (0-8): 0.0
Opacity (0-8): 0.0
Severity of the virus in  COVID-1653.jpg  X-Ray
Geographic Extent (0-8): 3.8
Opacity (0-8): 1.816
Severity of the virus in  COVID-1699.jpg  X-Ray
Geographic Extent (0-8): 4.824
Opacity (0-8): 5.278
Severity of the virus in  COVID-1723.jpg  X-Ray
Geographic Extent (0-8): 3.433
Opacity (0-8): 1.586
Severity of the virus in  COVID-1731.jpg  X-Ray
Geographic Extent (0-8): 0.9822
Opacity (0-8): 0.0
Severity of the virus in  COVID-1732.jpg  X-Ray
Geographic Extent (0-8): 6.086
Opacity (0-8): 4.55
Severity of the virus in  COVID-1735.jpg  X-Ray
Geographic Extent (0-8): 5.756
Opacity (0-8): 4.32
Severity of the virus in  COVID-1736.jpg  X-Ray
Geographic Extent (0-8): 6.776
Opacity (0-8): 3.